In [ ]:
https://msdn.microsoft.com/en-us/magazine/mt833293.aspx

In [ ]:
https://medium.com/coinmonks/create-a-neural-network-in-pytorch-and-make-your-life-simpler-ec5367895199

In [23]:
# import package
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

# set numpy seed for reproducibility
np.random.seed(seed=123)

In [24]:
dataset = pd.read_csv("../data/boston.csv", sep=',')[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat', 'medv']]
dataset.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [25]:
dataset.dtypes

crim       float64
zn         float64
indus      float64
chas         int64
nox        float64
rm         float64
age        float64
dis        float64
rad          int64
tax          int64
ptratio    float64
black      float64
lstat      float64
medv       float64
dtype: object

In [3]:
dataset = dataset.values
dataset[:2, :]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
        1.5300e+01, 3.9690e+02, 4.9800e+00, 2.4000e+01],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00, 2.1600e+01]])

## Split train- Test

In [5]:
train_size = 0.5
train_size = int(len(dataset) * train_size)
np.random.shuffle(dataset)
train, test = dataset[:train_size,:], dataset[train_size:,:]
print ('Dataset size: {0}. Train size: {1} - Test size: {2}'.format(len(dataset), len(train), len(test)))

Dataset size: 506. Train size: 253 - Test size: 253


In [6]:
# get train and set data into numpy nd.array
x_train = train[:,:-1]
x_test = test[:,:-1]
y_train = train[:,-1]
y_test = test[:,-1]

In [11]:
batch_size = len(x_train) # 50
num_epochs = 1000
learning_rate = 0.01
size_hidden = 100
n_feature = 13
n_output = 1
batch_no = int(len(x_train) / batch_size)

In [12]:
batch_no

1

In [17]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :",device)
class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden_1 = torch.nn.Linear(n_feature, size_hidden)   # hidden layer
        self.hidden_2 = torch.nn.Linear(size_hidden, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.sigmoid(self.hidden_1(x))      # activation function for hidden layer
        x = F.sigmoid(self.hidden_2(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x



net = Net(n_feature, size_hidden, n_output)

"""
net = torch.nn.Sequential(
    torch.nn.Linear(n_feature, size_hidden),
    torch.nn.Sigmoid(),
    torch.nn.Linear(size_hidden, n_output),
)
"""

Executing the model on : cpu


'\nnet = torch.nn.Sequential(\n    torch.nn.Linear(n_feature, size_hidden),\n    torch.nn.Sigmoid(),\n    torch.nn.Linear(size_hidden, n_output),\n)\n'

In [18]:

criterion = torch.nn.MSELoss(reduction='sum')

In [19]:
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
#criterion = torch.nn.MSELoss(size_average=False)  # this is for regression mean squared loss
criterion = torch.nn.MSELoss(reduction='sum')

In [20]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0

#inputs = Variable(torch.tensor(x_train).float())
#labels = Variable(torch.tensor(y_train).float())

batch_no

for epoch in range(num_epochs):
    
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.tensor(x_train[start:end]).float())
        labels = Variable(torch.tensor(y_train[start:end]).float())
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(outputs, torch.unsqueeze(labels, dim=1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0


Epoch 1 loss:  150410.609375
Epoch 2 loss:  137939.109375
Epoch 3 loss:  126483.4140625
Epoch 4 loss:  116403.6328125
Epoch 5 loss:  107722.9921875
Epoch 6 loss:  100246.984375
Epoch 7 loss:  93654.171875
Epoch 8 loss:  87638.28125
Epoch 9 loss:  81977.375
Epoch 10 loss:  76602.46875
Epoch 11 loss:  71501.5
Epoch 12 loss:  66686.078125
Epoch 13 loss:  62169.359375
Epoch 14 loss:  57964.015625
Epoch 15 loss:  54042.890625
Epoch 16 loss:  50370.68359375
Epoch 17 loss:  46931.671875
Epoch 18 loss:  43731.2890625
Epoch 19 loss:  40783.27734375
Epoch 20 loss:  38086.359375
Epoch 21 loss:  35631.32421875
Epoch 22 loss:  33409.42578125
Epoch 23 loss:  31418.810546875
Epoch 24 loss:  29650.369140625
Epoch 25 loss:  28087.447265625
Epoch 26 loss:  26715.1484375
Epoch 27 loss:  25525.00390625
Epoch 28 loss:  24511.166015625
Epoch 29 loss:  23663.97265625
Epoch 30 loss:  22968.80078125
Epoch 31 loss:  22412.486328125
Epoch 32 loss:  21982.546875
Epoch 33 loss:  21664.6015625
Epoch 34 loss:  21441

Epoch 260 loss:  4087.367431640625
Epoch 261 loss:  4077.43408203125
Epoch 262 loss:  4086.719482421875
Epoch 263 loss:  4081.40625
Epoch 264 loss:  4095.349365234375
Epoch 265 loss:  4051.682861328125
Epoch 266 loss:  4021.933837890625
Epoch 267 loss:  3952.135498046875
Epoch 268 loss:  3913.568603515625
Epoch 269 loss:  3898.241455078125
Epoch 270 loss:  3901.692138671875
Epoch 271 loss:  3923.19873046875
Epoch 272 loss:  3900.663818359375
Epoch 273 loss:  3913.958984375
Epoch 274 loss:  3876.8115234375
Epoch 275 loss:  3845.514892578125
Epoch 276 loss:  3795.79931640625
Epoch 277 loss:  3794.5966796875
Epoch 278 loss:  3891.339111328125
Epoch 279 loss:  4252.53466796875
Epoch 280 loss:  4268.76318359375
Epoch 281 loss:  3791.835693359375
Epoch 282 loss:  4339.84912109375
Epoch 283 loss:  4407.80712890625
Epoch 284 loss:  4120.7314453125
Epoch 285 loss:  4420.09521484375
Epoch 286 loss:  4256.6845703125
Epoch 287 loss:  4413.9033203125
Epoch 288 loss:  4115.5224609375
Epoch 289 loss:

Epoch 510 loss:  2268.181396484375
Epoch 511 loss:  2293.474853515625
Epoch 512 loss:  2257.363037109375
Epoch 513 loss:  2237.282958984375
Epoch 514 loss:  2201.90185546875
Epoch 515 loss:  2184.5849609375
Epoch 516 loss:  2174.304931640625
Epoch 517 loss:  2168.22802734375
Epoch 518 loss:  2173.481689453125
Epoch 519 loss:  2174.379638671875
Epoch 520 loss:  2193.025390625
Epoch 521 loss:  2195.571044921875
Epoch 522 loss:  2239.461181640625
Epoch 523 loss:  2252.49853515625
Epoch 524 loss:  2372.90869140625
Epoch 525 loss:  2307.03857421875
Epoch 526 loss:  2187.495361328125
Epoch 527 loss:  2164.221435546875
Epoch 528 loss:  2131.458251953125
Epoch 529 loss:  2158.494873046875
Epoch 530 loss:  2136.200927734375
Epoch 531 loss:  2125.039306640625
Epoch 532 loss:  2112.86328125
Epoch 533 loss:  2173.14306640625
Epoch 534 loss:  2220.162109375
Epoch 535 loss:  2162.458251953125
Epoch 536 loss:  2634.70849609375
Epoch 537 loss:  2807.440673828125
Epoch 538 loss:  2898.534423828125
Epoc

Epoch 746 loss:  1520.3544921875
Epoch 747 loss:  1518.8619384765625
Epoch 748 loss:  1517.391845703125
Epoch 749 loss:  1515.9381103515625
Epoch 750 loss:  1514.4979248046875
Epoch 751 loss:  1513.0677490234375
Epoch 752 loss:  1511.64599609375
Epoch 753 loss:  1510.231689453125
Epoch 754 loss:  1508.82373046875
Epoch 755 loss:  1507.4176025390625
Epoch 756 loss:  1506.0113525390625
Epoch 757 loss:  1504.6036376953125
Epoch 758 loss:  1503.19384765625
Epoch 759 loss:  1501.7774658203125
Epoch 760 loss:  1500.3482666015625
Epoch 761 loss:  1498.8980712890625
Epoch 762 loss:  1497.4161376953125
Epoch 763 loss:  1495.887939453125
Epoch 764 loss:  1494.3011474609375
Epoch 765 loss:  1492.6322021484375
Epoch 766 loss:  1490.83544921875
Epoch 767 loss:  1488.8460693359375
Epoch 768 loss:  1486.5806884765625
Epoch 769 loss:  1483.918212890625
Epoch 770 loss:  1480.6927490234375
Epoch 771 loss:  1476.7418212890625
Epoch 772 loss:  1472.3082275390625
Epoch 773 loss:  1467.22509765625
Epoch 774

Epoch 988 loss:  1197.515625
Epoch 989 loss:  1190.0186767578125
Epoch 990 loss:  1189.7811279296875
Epoch 991 loss:  1189.118896484375
Epoch 992 loss:  1184.5989990234375
Epoch 993 loss:  1185.5382080078125
Epoch 994 loss:  1183.0181884765625
Epoch 995 loss:  1182.2115478515625
Epoch 996 loss:  1182.1405029296875
Epoch 997 loss:  1180.2120361328125
Epoch 998 loss:  1180.5791015625
Epoch 999 loss:  1181.13037109375
Epoch 1000 loss:  1184.299560546875


In [21]:
type(x_train[start:end])

numpy.ndarray

In [22]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

X = Variable(torch.tensor(x_train).float()) 
result = net(X)
pred=result.data[:,0].numpy()
print(len(pred),len(y_train))
print (r2_score(pred,y_train))
print (mean_squared_error(pred,y_train))
print (np.mean(y_train))

253 253
0.9412235092713305
4.7024455087249555
22.636758893280636


In [185]:
np.mean(np.sqrt((pred - y_train)**2))

2.3162933756711457

In [173]:
np.sq(4)

array([], dtype=float64)

In [180]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

X = Variable(torch.tensor(x_test).float()) 
result = net(X)
pred=result.data[:,0].numpy()
print(len(pred),len(y_test))
print (r2_score(pred,y_test))
print (mean_squared_error(pred,y_test))
print (np.mean(y_test))

253 253
0.7635263966826227
16.128128208921893
22.231620553359686


## Create model
Then we define the sizes of all the layers and the batch size

In [8]:
n_in, n_h1, n_out, batch_size = 13, 5, 1, len(tensor_x_train)

In [9]:
n_feature, size_hidden, n_output = 13, 5, 1

In [16]:
batch_no = len(x_train)
batch_size = 50
num_epochs = 200
learning_rate = 0.01
size_hidden= 100

In [45]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :", device)

class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
net = Net(n_feature, size_hidden, n_output)

Executing the model on : cpu


In [46]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [47]:
criterion = torch.nn.MSELoss(reduction='sum')

In [48]:
from torch.autograd import Variable
running_loss = 0.0
for epoch in range(num_epochs):
    # Mini batch learning
    #for i in range(50):
    inputs = Variable(torch.FloatTensor(x_train))
    labels = Variable(torch.FloatTensor(y_train))
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    #print("outputs",outputs)
    #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
    loss = criterion(outputs, torch.unsqueeze(labels, dim=1))
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0

Epoch 1 loss:  189120.421875
Epoch 2 loss:  4.15542021528299e+23
Epoch 3 loss:  inf
Epoch 4 loss:  nan
Epoch 5 loss:  nan
Epoch 6 loss:  nan
Epoch 7 loss:  nan
Epoch 8 loss:  nan
Epoch 9 loss:  nan
Epoch 10 loss:  nan
Epoch 11 loss:  nan
Epoch 12 loss:  nan
Epoch 13 loss:  nan
Epoch 14 loss:  nan
Epoch 15 loss:  nan
Epoch 16 loss:  nan
Epoch 17 loss:  nan
Epoch 18 loss:  nan
Epoch 19 loss:  nan
Epoch 20 loss:  nan
Epoch 21 loss:  nan
Epoch 22 loss:  nan
Epoch 23 loss:  nan
Epoch 24 loss:  nan
Epoch 25 loss:  nan
Epoch 26 loss:  nan
Epoch 27 loss:  nan
Epoch 28 loss:  nan
Epoch 29 loss:  nan
Epoch 30 loss:  nan
Epoch 31 loss:  nan
Epoch 32 loss:  nan
Epoch 33 loss:  nan
Epoch 34 loss:  nan
Epoch 35 loss:  nan
Epoch 36 loss:  nan
Epoch 37 loss:  nan
Epoch 38 loss:  nan
Epoch 39 loss:  nan
Epoch 40 loss:  nan
Epoch 41 loss:  nan
Epoch 42 loss:  nan
Epoch 43 loss:  nan
Epoch 44 loss:  nan
Epoch 45 loss:  nan
Epoch 46 loss:  nan
Epoch 47 loss:  nan
Epoch 48 loss:  nan
Epoch 49 loss:  nan
Ep

In [41]:
import pandas as pd
from sklearn.metrics import r2_score

X = Variable(torch.FloatTensor(x_train)) 
result = net(X)
pred=result.data[:,0].numpy()
print(len(pred),len(y_train))
r2_score(pred,y_train)

354 354


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [42]:
labels

tensor([15.0000, 26.6000, 45.4000, 20.8000, 34.9000, 21.9000, 28.7000,  7.2000,
        20.0000, 32.2000, 24.1000, 18.5000, 13.5000, 27.0000, 23.1000, 18.9000,
        24.5000, 43.1000, 19.8000, 13.8000, 15.6000, 50.0000, 37.2000, 46.0000,
        50.0000, 21.2000, 14.9000, 19.6000, 19.4000, 18.6000, 26.5000, 32.0000,
        10.9000, 20.0000, 21.4000, 31.0000, 25.0000, 15.4000, 13.1000, 37.6000,
        37.0000, 18.9000, 27.9000, 50.0000, 14.4000, 22.0000, 19.9000, 21.6000,
        15.6000, 15.0000, 32.4000, 29.6000, 20.4000, 12.3000, 19.1000, 14.9000,
        17.8000,  8.8000, 35.4000, 11.5000, 19.6000, 20.6000, 15.6000, 19.9000,
        23.3000, 22.3000, 24.8000, 16.1000, 22.8000, 30.5000, 20.4000, 24.4000,
        16.6000, 26.2000, 16.4000, 20.1000, 13.9000, 19.4000, 22.8000, 13.8000,
        31.6000, 10.5000, 23.8000, 22.4000, 19.3000, 22.2000, 12.6000, 19.4000,
        22.2000, 29.8000,  9.6000, 34.9000, 21.4000, 25.3000, 32.9000, 26.6000,
        14.6000, 31.5000, 23.3000, 33.30

In [43]:
net(inputs)

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

In [98]:
model(tensor_x_train)

tensor([[22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.3813],
        [22.